In [1]:
from MetricRefresh import IncrementalICCalculator, IncrementalMonotonic, IncrementalMI, IncrementalFvalue, OscillationFromSpread, SpreadReturns
from Database import Database
from GetSignal import GetSignal

In [2]:
from google.cloud import bigquery

def get_unique_signals_from_gbq(project_id='issachar-feature-library', 
                                     dataset_name='wmg', 
                                     table_name='signal_lookup'):
    """
    Fetch all unique signal values from the 'signal' column in the daily_ic table.
    
    Args:
        project_id (str): The Google Cloud project ID.
        dataset_name (str): The BigQuery dataset name.
        table_name (str): The BigQuery table name containing the daily IC results.
    
    Returns:
        list: A list of unique signal names.
    """
    # Create a BigQuery client.
    client = bigquery.Client(project=project_id)
    
    # Construct the query to select distinct signals.
    query = f"""
    SELECT DISTINCT signal_name
    FROM `{project_id}.{dataset_name}.{table_name}`
    WHERE signal_type != 'neither'
    """
    
    # Execute the query.
    query_job = client.query(query)
    
    # Convert the result to a pandas DataFrame.
    df = query_job.to_dataframe()
    
    # Return the unique signals as a list.
    return df['signal_name'].unique().tolist()

In [3]:
# Pull all signals from GBQ
all_signals = get_unique_signals_from_gbq()
all_signals[:5]

['adlr_21d',
 'ptx_est_rev_180d',
 'buyer_pos_12m',
 'insider_ssells_3m',
 'seller_chge_6m']

In [4]:
# 1) Create Database
db = Database(project_id="issachar-feature-library")

# 2) Create GetSignal
get_signal = GetSignal(db=db)

In [6]:
for signal_name in tqdm(all_signals):

    # Create IncrementalICCalculator
    ic_calc = IncrementalICCalculator(
        db=db, 
        get_signal=get_signal, 
        project_id="issachar-feature-library",
        wmg_dataset="wmg",
        returns_table="t1_returns",
        daily_ic2_table="daily_ic2"
    )

    # Compute incremental IC
    ic_calc.run(signal_name, rank_signal=True, refresh_data = False)    

    # Compute Incremental monotonicity
    mon_calc = IncrementalMonotonic(
        db=db,
        get_signal=get_signal,
        project_id="issachar-feature-library",
        wmg_dataset="wmg",
        returns_table="t1_returns",
        monotonic_table="daily_rolling_monotonic_pct_st"
    )
    mon_calc.run(signal_name, refresh_data = False)

    # Create IncrementalMI
    mi_calc = IncrementalMI(
        db=db,
        get_signal=get_signal,
        project_id="issachar-feature-library",
        wmg_dataset="wmg",
        returns_table="t1_returns",
        table_daily_mi="mi_daily",
        table_rolling_mi="mi_rolling",
        window_size=42
    )

    # Compute incremental MI for a single signal
    mi_calc.run(signal_name, refresh_data = False)    
    

    # Fvalue related stats 
    fval_calc = IncrementalFvalue(
        db=db,
        get_signal=get_signal,
        project_id="issachar-feature-library",
        wmg_dataset="wmg",
        returns_table="t1_returns",
        table_daily_fval="daily_fvalues",
        table_rolling_fval="daily_fvalue_interactions"  # if you want the name from snippet
    )

    fval_calc.run(signal_name, refresh_data=False)    
    
    # Calc spread returns
    spread_calc = SpreadReturns(
        db=db,                 # your Database instance
        get_signal=get_signal, # your GetSignal instance
        project_id="issachar-feature-library",
        wmg_dataset="wmg",
        daily_spread_table="daily_spread_returns",  # The BQ table to store final results
        returns_table="t1_returns"                  # Where we get t1_returns
    )

    spread_calc.run(signal_name, refresh_data=False)  
    

    # Calc oscillation metrics
    osc_calc = OscillationFromSpread(
        project_id="issachar-feature-library",
        dataset_name="wmg",
        daily_spread_table="daily_spread_returns",
        oscillation_table="daily_oscillations_90d"
    )

    # Suppose we want to do "accel_21d" with daily_spread
    osc_calc.run(signal_name)

## Scratch code from testing

In [ ]:
# from google.cloud import bigquery

# def fetch_ic2_after_sep1_2024(db, signal_name: str) -> None:
#     """
#     Fetch rows from daily_ic2 for the given signal_name
#     where date > '2024-09-01'.
#     Print the results or return them as a DataFrame.
#     """
#     client = bigquery.Client(project=db.project_id)
    
#     query = """
#     SELECT date,
#            all_IC, all_tstat, all_pvalue, all_fstat,
#            all_IC_spearmanr, all_pvalue_spearmanr,
#            topq_IC, topq_tstat, topq_pvalue, topq_fstat,
#            topq_IC_spearmanr, topq_pvalue_spearmanr,
#            signal
#     FROM `issachar-feature-library.wmg.daily_ic2`
#     WHERE signal = @signal
#       AND date > @cutoff_date
#     """
    
#     job_config = bigquery.QueryJobConfig(
#         query_parameters=[
#             bigquery.ScalarQueryParameter("signal", "STRING", signal_name),
#             # If your table's date column is TIMESTAMP, pass TIMESTAMP here.
#             # If it's DATETIME, pass DATETIME. We'll assume TIMESTAMP below:
#             bigquery.ScalarQueryParameter("cutoff_date", "DATE", "2024-09-01")
#         ]
#     )
    
#     df = client.query(query, job_config=job_config).to_dataframe()
#     print(f"Fetched {len(df)} rows for signal='{signal_name}' with date > 2024-09-01.")
#     print(df.head(10))  # Example: print first 10 rows


# df_ic = fetch_ic2_after_sep1_2024(db, "daysToCover30Day")
# df_ic

In [ ]:
# GetSignal(db).run(signal_name, end_date = None)

In [ ]:
# ic_calc._get_daily_ic2_minmax_date(signal_name)

In [ ]:
# from google.cloud import bigquery

# def delete_daily_ic2_for_daysToCover30Day():
#     """
#     Deletes rows in 'issachar-feature-library.wmg.daily_ic2' 
#     where signal='daysToCover30Day' AND date > '2024-09-01'.
#     """
#     client = bigquery.Client(project="issachar-feature-library")

#     query = """
#     DELETE FROM `issachar-feature-library.wmg.daily_oscillations_90d`
#     WHERE signal = @sig
#       AND DATE(date) > @cutoff_date
#     """
#     job_config = bigquery.QueryJobConfig(
#         query_parameters=[
#             bigquery.ScalarQueryParameter("sig", "STRING", "daysToCover30Day"),
#             bigquery.ScalarQueryParameter("cutoff_date", "DATE", "2024-09-01"),
#         ]
#     )

#     client.query(query, job_config=job_config).result()
#     print("Rows deleted for 'daysToCover30Day' in daily_ic2 after 2024-09-01.")

# # Usage example:
# delete_daily_ic2_for_daysToCover30Day()
